In [ ]:
import uuid

from rich.pretty import pprint

from langchain_openai import OpenAIEmbeddings
from langchain_qdrant import QdrantVectorStore
from langchain_core.documents import Document

from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

In [ ]:
# NOTE: https://python.langchain.com/docs/integrations/vectorstores/qdrant/
EMBEDDING_DIMS = 256

In [ ]:
client = QdrantClient(":memory:")
client.create_collection(
    collection_name="demo_collection",
    vectors_config=VectorParams(
        size=EMBEDDING_DIMS,
        distance=Distance.COSINE,
    ),
)

In [ ]:
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-large",
    dimensions=EMBEDDING_DIMS,
)

vector_store = QdrantVectorStore(
    client=client,
    collection_name="demo_collection",
    embedding=embeddings,
)

In [ ]:
documents = [
    Document(
        page_content="I had chocolate chip pancakes and scrambled eggs for breakfast this morning.",
        metadata={
            "source": "tweet",
        },
    ),
    Document(
        page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees Fahrenheit.",
        metadata={
            "source": "news",
        },
    ),
    Document(
        page_content="Robbers broke into the city bank and stole $1 million in cash.",
        metadata={
            "source": "news",
        },
    ),
    Document(
        page_content="Wow! That was an amazing movie. I can't wait to see it again.",
        metadata={
            "source": "tweet",
        },
    ),
]

uuids = [str(uuid.uuid4()) for _ in range(len(documents))]
_ = vector_store.add_documents(
    documents=documents,
    ids=uuids,
)

In [ ]:
results = vector_store.similarity_search_with_score("Movies", k=2)
pprint(results)